In [1]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset, TensorDataset
import os

# Check for available devices
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define normalization dictionary
NORMALIZE_DICT = {
    'cifar10': dict(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
}


Using device: cuda:0


In [2]:
# Dataset name and directory
dataset = "cifar10"
data_dir = './data'

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR10(data_dir, train=True, download=True,
                                  transform=transforms.Compose([
                                      transforms.ToTensor(),
                                  ]))
test_dataset = datasets.CIFAR10(data_dir, train=False,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
                                 ]))

# Merge training and testing datasets
merged_dataset = ConcatDataset([train_dataset, test_dataset])
merged_loader = DataLoader(merged_dataset, batch_size=32, shuffle=False)

# Collect all images and labels
all_data, all_targets = [], []
for images, labels in merged_loader:
    all_data.append(images)
    all_targets.append(labels)

merged_data = torch.cat(all_data, dim=0)
merged_targets = torch.cat(all_targets, dim=0)

In [3]:
# Function to generate Dirichlet-distributed partitions
def dirichlet_distribution(alpha, num_clients):
    return np.random.dirichlet([alpha] * num_clients)

# Function to partition dataset using Dirichlet distribution
def partition_dataset_dirichlet(data, targets, num_clients, alpha, min_samples=100):
    num_classes = len(set(targets.numpy()))
    data_indices = [[] for _ in range(num_clients)]
    labels = np.array(targets)

    while True:  # Ensure minimum number of samples per client
        data_indices = [[] for _ in range(num_clients)]

        for k in range(num_classes):
            class_indices = np.where(labels == k)[0]
            np.random.shuffle(class_indices)

            # Dirichlet-based class distribution across clients
            proportions = dirichlet_distribution(alpha, num_clients)
            proportions = (proportions / proportions.sum()) * len(class_indices)
            proportions = np.cumsum(proportions).astype(int)

            previous_idx = 0
            for i in range(num_clients):
                client_indices = class_indices[previous_idx:proportions[i]]
                data_indices[i].extend(client_indices)
                previous_idx = proportions[i]

        # Ensure each client has at least `min_samples`
        if all(len(indices) >= min_samples for indices in data_indices):
            break

    return data_indices

In [4]:
# Function to apply data augmentation
def apply_augmentation(data):
    transform = transforms.Compose([
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.Resize((32, 32)),
        transforms.Normalize(**NORMALIZE_DICT[dataset]),
        transforms.Lambda(lambda x: x + 0.01 * torch.randn_like(x))  # Small Gaussian noise
    ])
    return torch.stack([transform(img) for img in data])

# Function to save partitioned datasets
def save_partitioned_data(data, targets, data_indices):
    for i, indices in enumerate(data_indices):
        subset_data = apply_augmentation(data[indices])
        subset_targets = targets[indices]

        filename = f"client_{i+1}.pt"
        torch.save({'data': subset_data, 'targets': subset_targets}, filename)
        print(f"Saved {filename} with {len(indices)} samples.")

# Number of clients and Dirichlet parameter
num_clients = 70  # Example: Partition dataset among 10 clients
alpha = 0.7  # Controls non-IID level

# Partition dataset
data_indices = partition_dataset_dirichlet(merged_data, merged_targets, num_clients, alpha)

# Save partitioned datasets
save_partitioned_data(merged_data, merged_targets, data_indices)

Saved client_1.pt with 1110 samples.
Saved client_2.pt with 1421 samples.
Saved client_3.pt with 736 samples.
Saved client_4.pt with 1121 samples.
Saved client_5.pt with 1170 samples.
Saved client_6.pt with 1373 samples.
Saved client_7.pt with 893 samples.
Saved client_8.pt with 366 samples.
Saved client_9.pt with 750 samples.
Saved client_10.pt with 906 samples.
Saved client_11.pt with 731 samples.
Saved client_12.pt with 676 samples.
Saved client_13.pt with 579 samples.
Saved client_14.pt with 622 samples.
Saved client_15.pt with 285 samples.
Saved client_16.pt with 917 samples.
Saved client_17.pt with 731 samples.
Saved client_18.pt with 911 samples.
Saved client_19.pt with 1046 samples.
Saved client_20.pt with 701 samples.
Saved client_21.pt with 890 samples.
Saved client_22.pt with 910 samples.
Saved client_23.pt with 1002 samples.
Saved client_24.pt with 966 samples.
Saved client_25.pt with 1182 samples.
Saved client_26.pt with 835 samples.
Saved client_27.pt with 435 samples.
Sa